In [ ]:
# !pip install datasets evaluate transformers[sentencepiece]
# !pip install accelerate
# # To run the training on TPU, you will need to uncomment the following line:
# # !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# !apt install git-lfs
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
from datasets import load_dataset

from datasets import load_from_disk

load_dir = "../data/my_data_NO_ZERO"

raw_datasets = load_from_disk(load_dir)
print(raw_datasets)
raw_datasets['train'][0]
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs
tokenized_dataset = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets["train"].column_names)
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoConfig

config = AutoConfig.from_pretrained('distilbert-base-uncased')
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased").to('cuda')
training_args = TrainingArguments(
    output_dir="span_nz",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)

trainer.train()
model.save_pretrained("../models/span_nz")
tokenizer.save_pretrained("../models/span_nz")
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoConfig

model = AutoModelForQuestionAnswering.from_pretrained("../models/span_nz").to('cuda')
raw_datasets['validation']['question']
from transformers import pipeline

question_answerer = pipeline("question-answering", model="../models/span_nz", tokenizer="../models/span_nz", device=device)
predictions = question_answerer(question=raw_datasets['validation']['question'], context=raw_datasets['validation']['context'])
predictions
import evaluate

def findEvalMetrics(true_labels, predictions):
    bleu = evaluate.load("bleu")
    results_bleu1 = bleu.compute(predictions=predictions, references=true_labels,max_order = 1)
    results_bleu2 = bleu.compute(predictions=predictions, references=true_labels,max_order = 2)
    results_bleu3 = bleu.compute(predictions=predictions, references=true_labels,max_order = 3)
    results_bleu4 = bleu.compute(predictions=predictions, references=true_labels,max_order = 4)

    results_bleu=[results_bleu1, results_bleu2, results_bleu3, results_bleu4]

    meteor = evaluate.load("meteor")
    results_meteor = meteor.compute(predictions=predictions, references=true_labels)

    return [results_bleu,results_meteor]
# true_labels = [x['text'][0] if x['text'][0] != '' else 'null' for x in raw_datasets['validation']['answers']]
true_labels = [x['text'][0] for x in raw_datasets['validation']['answers']]
true_labels
# pred_labels = [x['answer'] if len(x['answer'].split()) != 1 else 'null' for x in predictions]
# pred_labels = [x['answer'] if len(x['answer'].split()) != 1 else '' for x in predictions]
pred_labels = [x['answer'] for x in predictions]
pred_labels
metrics = findEvalMetrics(true_labels, pred_labels)
metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='weighted', zero_division=1)
recall = recall_score(true_labels, pred_labels, average='weighted', zero_division=1)
f1 = f1_score(true_labels, pred_labels, average='weighted', zero_division=1)
accuracy, precision, recall, f1
context = 'all right look you are not really gonna buy that are you do not you think you have embarrassed me enough for one day'
question = 'oh i embarrass you'
ans = 'you have embarrassed me enough for one day'
question = 'Ohh , you are about to get a little luckier .'
context = 'You look amazing . I am the luckiest man in the world .'
answer = 'You look amazing . I am the luckiest man in the world .'
from transformers import pipeline

question_answerer = pipeline("question-answering", model="../models/span_nz", tokenizer="../models/span_nz", device=device)
test_predictions = question_answerer(question=question, context=context)
test_predictions
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_qa_model")
inputs = tokenizer(question, context, return_tensors="pt")
import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("my_awesome_qa_model")
with torch.no_grad():
    outputs = model(**inputs)
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)